In [1]:
# import os
# os.environ["GOOGLE_API_KEY"] = ""
# os.environ["ANTHROPIC_API_KEY"] = ""
# os.environ["LANGCHAIN_TRACING_V2"] = ""
# os.environ["LANGCHAIN_API_KEY"]=""

from dotenv import load_dotenv
load_dotenv()

True

## Prompts

In [2]:
environmental_template = """You are a passionate environmentalist with a deep understanding of ecological systems and sustainability principles. \
You excel at providing insightful explanations and solutions related to environmental issues in a clear and engaging manner. \
When faced with a question outside your expertise, you gracefully acknowledge your limitations.\

Answer the following question based on the provided context:

<context>
{context}
</context>

Here is the question:
{input}"""

education_template = """You are an enthusiastic and knowledgeable educator with a profound understanding of pedagogy and learning principles. \
You are adept at delivering comprehensive explanations and fostering engaging learning experiences. 

Answer the following question with the provided context.

<context>
{context}
</context>

Here is the question:
{input}"""

In [3]:
prompt_infos = [
{
    "name": "environmentalist",
    "description": "Ideal for addressing environmental concerns and offering insightful solutions",
    "prompt_template": environmental_template,
},
{
    "name": "education", 
    "description": "Good for answering questions related to education or study or 早自習", 
    "prompt_template": education_template,
}]

In [5]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
你是一個router，你的工作是分析input的問題與甚麼領域有關，\
當問題不清楚時適當修改問題，接著把問題傳遞給適當的language model.\
你會得到所有可以選擇的language model名稱及它們擅長哪個領域的問題。\

You can also use the history messages provided here to make your decision.
<< HISTORY >>
{{history}}

<< INPUT >>
{{input}}


<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:

\```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ summary of the history relevant to the topic of input
}}}}
\```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is completely\
unrelated to any of the candidate prompts.
REMEMBER: "next_inputs" should contain a concise summary of the history to provide context for the input.


<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

## LLM

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

gemini = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro", convert_system_message_to_human=True)
claude = ChatAnthropic(temperature=0,  model_name="claude-3-haiku-20240307")
# openai = ChatOpenAI()

c:\Users\ethan\miniconda3\envs\genai-course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document loader & text splitter

In [7]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = Chroma.from_documents(documents, embedding_function)

In [ ]:
vector.delete_collection()

## Destinations

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_template(template=environmental_template)

document_chain = create_stuff_documents_chain(claude, prompt)
retriever = vector.as_retriever(search_kwargs={"k": 1})

enviromental_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
prompt = ChatPromptTemplate.from_template(template=education_template)

document_chain = create_stuff_documents_chain(claude, prompt)
retriever = vector.as_retriever(search_kwargs={"k": 1})

education_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

general_chain = RunnablePassthrough.assign(
    answer=(ChatPromptTemplate.from_template("{history} {input}") | claude | StrOutputParser())
)

## Route to Destination

In [12]:
def route(info):
    print(info)
    if info["destination"]:
        if "environmentalist" in info["destination"].lower():
            return enviromental_chain
        elif "education" in info["destination"].lower():
            return education_chain
    else:
        return general_chain

In [13]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# destinations
# ['physics: Good for answering questions about physics',
#  'math: Good for answering math questions']

router_system_template = PromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)
router_system_template = router_system_template.format(destinations=destinations_str)

router_system_template = HumanMessagePromptTemplate.from_template(router_system_template)

## Memory 

In [14]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(llm=gemini, max_token_limit=1024, ai_prefix="")
memory.load_memory_variables({})

{'history': ''}

In [15]:
router_prompt = ChatPromptTemplate.from_messages(
    [
        # MessagesPlaceholder(variable_name="history", optional=True), # Put history into router llm ???
        router_system_template,
    ]
)

## Build Chain

In [16]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.router.llm_router import RouterOutputParser
from operator import itemgetter

router_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | router_prompt
    | claude
    | RouterOutputParser()
    | {"destination": itemgetter("destination"), "next_inputs": lambda x: x["next_inputs"]['input']} 
)

In [17]:

full_chain = (
    RunnablePassthrough.assign(
        route=router_chain
    )
    | { 
        "destination": lambda x: x["route"]["destination"], 
        "input": lambda x: f'{x["route"]["next_inputs"]}\n{x["input"]}'
    } 
    | RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    ) 
    | RunnableLambda(route)
) 

## Usage

In [18]:
from api.server import Server
from api.manager import Manager
from api.models.match import Match

server = Server("http://localhost:3000", "0929b6b4-1657-41db-a109-c33bc23abbb0")
manager = Manager(server)

In [19]:
print("all matches:")
matches = manager.get_match_list()
for match in matches:
    print(match)
print("-" * 80)
manager.set_current_match(2)

all matches:
Match ID: 1, Name: testgame, Players: [], History Messages: []
Match ID: 2, Name: testgame, Players: [], History Messages: []
--------------------------------------------------------------------------------


In [20]:
print("current history:")
match = manager.get_current_match()
for msg in match.history_msgs:
    print(msg)

current history:


In [21]:
memory.clear()

In [22]:
inputs = []

inputs.append({"input": 
"""兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?
正方：支持廢除早自習
反方：反對廢除早自習

作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。
正方:"""})

inputs.append({"input": 
"""作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。
反方:"""
})

inputs.append({"input": 
"""作為正方，請堅守支持廢除早自習的立場。請針對反對方所發表的意見，進行反駁。
正方:"""})

inputs.append({"input": 
"""作為反方，請堅守反對廢除早自習的立場。請針對正方所發表的意見，進行反駁。
反方:"""})

In [24]:
print(inputs)

[{'input': '兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?\n正方：支持廢除早自習\n反方：反對廢除早自習\n\n作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。\n正方:'}, {'input': '作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。\n反方:'}, {'input': '作為正方，請堅守支持廢除早自習的立場。請針對反對方所發表的意見，進行反駁。\n正方:'}, {'input': '作為反方，請堅守反對廢除早自習的立場。請針對正方所發表的意見，進行反駁。\n反方:'}]


In [30]:
import time

local_match_progress = 0
first = True

while True:

    match = manager.get_current_match()

    # init 
    if local_match_progress == 0 and first:
        # respond
        current_input = inputs[local_match_progress]
        response = full_chain.invoke(current_input)
        answer = response.get("answer")
        
        # save 
        print(answer)
        memory.save_context(current_input, {"output": answer})
        local_match_progress += 1
        
        if (local_match_progress == 4):
            print("End")
            break

        # push
        manager.send_message(answer)

    # lag behind remote history: sync with server
    if local_match_progress < len(match.history_msgs):
        assert local_match_progress == len(match.history_msgs) - 1

        # pull
        new_messages = match.history_msgs[local_match_progress]
        print(new_messages.text)

        # save
        current_input = inputs[local_match_progress]
        memory.save_context(current_input, {"output": new_messages.text})
        local_match_progress += 1

        if (local_match_progress == 4):
            print("End")
            break
        
        # respond
        current_input = inputs[local_match_progress]
        response = full_chain.invoke(current_input)
        answer = response.get("answer")

        # save 
        print(answer)
        memory.save_context(current_input, {"output": answer})
        local_match_progress += 1

        # push
        manager.send_message(answer)

        if (local_match_progress == 4):
            print("End")
            break

    time.sleep(5)
    

{'destination': 'education', 'input': 'The input is about a debate on whether to abolish early self-study (早自習) in schools. The affirmative side (正方) argues in favor of abolishing early self-study, while the negative side (反方) opposes abolishing it. The affirmative side has presented their arguments supporting the abolition of early self-study.\n兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?\n正方：支持廢除早自習\n反方：反對廢除早自習\n\n作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。\n正方:', 'history': 'Human: 兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?\n正方：支持廢除早自習\n反方：反對廢除早自習\n\n作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。\n正方:\n: 尊敬的觀眾朋友們,大家好!作為支持廢除早自習的正方,我們有以下幾點理由:\n\n首先,早自習對學生的身心健康造成了負面影響。長時間的早起和自主學習,會導致學生睡眠不足、精神疲憊,影響學習效率和課堂表現。過度的學習壓力也可能引發焦慮和抑鬱等心理問題。我們應該關注學生的整體發展,而不是單純追求成績。\n\n其次,早自習並不能真正提高學習效果。許多學生在早自習時間缺乏集中注意力,效率低下。相反,適當的休息和娛樂活動,反而有助於提高學習興趣和效率。我們應該鼓勵學生養成良好的學習習慣,而不是強迫他們在早晨進行機械式的自主學習。\n\n最後,取消早自習可以為學生騰出更多自由時間,讓他們有機會參與其他有益的課外活動,如運動、音樂、社交等。這些活動不僅有助於身心健康,也有利於培養學生的全面素質。\n\n綜上所述,我們認為廢除早自習是一個明智的決定,能夠更好地照顧學生的需求,促進他們的全面發展。讓我們共同為學生創造更加健康、快樂的學習環境。\nHuman: 兩位

In [ ]:
inputs = {"input": 
"""兩位參賽者進行辯論比賽，今天的討論議題是支不支持廢除早自習?
正方：支持廢除早自習
反方：反對廢除早自習

作為正方，請堅守支持廢除早自習的立場，並簡明扼要地陳述正方的意見和理由。
正方:"""}
response = full_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為反方，請堅守反對廢除早自習的立場，簡明扼要地陳述你的意見和理由。
反方:"""
}
response = full_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為正方，請堅守支持廢除早自習的立場。請針對反對方所發表的意見，進行反駁。
正方:"""}
response = full_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": 
"""作為反方，請堅守反對廢除早自習的立場。請針對正方所發表的意見，進行反駁。
反方:"""}
response = full_chain.invoke(inputs)
print(response.get("answer"))
memory.save_context(inputs, {"output": response.get("answer")})

## Plot

In [ ]:
full_chain.get_graph().print_ascii()